In [1]:
from dotenv import load_dotenv
load_dotenv("dev.env")
import os

db_name = os.getenv("db_name")
db_username = os.getenv("db_username")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_port = os.getenv("db_port")
website = os.getenv("website")
city_link = os.getenv("city_link")

In [2]:
import psycopg2
import time
import math
import numpy as np
import os.path

conn_string = 'host={pghost} port={pgport} dbname={pgdatabase} user={pguser} password={pgpassword}'.format(pgdatabase=db_name,pguser=db_username,pgpassword=db_password,pghost=db_host,pgport=db_port)
conn=psycopg2.connect(conn_string)
cur=conn.cursor()

def check_if_table_exists(schema,table):
    cur.execute("select exists(select * from information_schema.tables where table_schema='{schema}' AND table_name='{table}')".format(schema=schema, table=table))
    return cur.fetchone()[0]

def check_if_index_exists(index):
    cur.execute("SELECT EXISTS(SELECT * FROM PG_CLASS WHERE relname = '{index}')".format(index=index))
    return cur.fetchone()[0]

def check_if_file_exists(filename):
    return os.path.isfile(filename)

def execute_mogrify(conn, df, schema, table):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = '"'+'","'.join(list(df.columns))+'"'
    cursor = conn.cursor()    
    try:
        for tup in tuples:
            query  = """INSERT INTO "{schema}"."{table}"({cols}) VALUES ({values})""".format(schema=schema,table=table, cols=cols, values=",".join(map(str,tup)))
            cursor.execute(query)
            conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

def df_column_conversation(df, column_name, type):
    if(type == 'timestamp'):
        df[column_name] = df[column_name].apply(lambda x: f"'{x}'::timestamp")
    if(type == 'text'):
        df[column_name] = df[column_name].apply(lambda x: f"'{x}'")
    if(type == 'date'):
        df[column_name] = df[column_name].apply(lambda x: f"'{x}'::date")
    if(type == 'numeric'):
        df[column_name] = df[column_name].str.replace(',','.')
    if(type == 'integer'):
        df[column_name] = df[column_name].str.replace(',','.').apply(float).apply(int)

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

city_req = requests.get(city_link)
city_soup = BeautifulSoup(city_req.content,"lxml")
for restaurant in city_soup.find(name="div", attrs={"class":"ys-reslist"}).find_all(name="a", attrs={"class":"restaurantName"}):
    restaurant_name = restaurant.span.text
    restaurant_sublink = restaurant.get("href")
    restaurant_link = website+restaurant_sublink
    print(restaurant_link)
    restaurant_req = requests.get(restaurant_link)
    restaurant_soup = BeautifulSoup(restaurant_req.content,"lxml")
    for category_soup in restaurant_soup.select("div[class*=restaurantDetailBox]"):
        try:
            category_name = category_soup.find(name="b").text
        except:
            continue
    print(restaurant_soup.find_all(name="div", attrs={"class":"listBody"}))
    t1 = restaurant_soup.find_all(name="div", attrs={"class":"listBody"})
    break  

https://www.yemeksepeti.com/pizza-dante-sisli-ergenekon-mah-pangalti-istanbul
</span> </b> </div> </div> </div>, <div class="listBody"> <div id="ys-approved-text"> <span class="ys-icons ys-icons-Standard-icons-info info-icon"></span> <span class="approve-text">Bu restoranın <strong>adres</strong> ve <strong>onaylanmış</strong> telefon bilgileri Yemeksepeti'nde bulunmaktadır.</span> </div> </div>]


In [4]:
restaurant_link

'https://www.yemeksepeti.com/pizza-dante-sisli-ergenekon-mah-pangalti-istanbul'

In [5]:
product_name = t1[0].find(name="a", attrs={"class":"getProductDetail"}).text
product_desc = t1[0].find(name="div", attrs={"class":"product-desc"}).text
#product_price = t1[0].select("div[class=product-price]")[0].select("span[class=price]")[0].text
product_price = t1[0].select("div[class*=product-price]")[0].find(name="span").text
try:
    #discount_price = t1[0].select("div[class=product-price discounted]")[0].select("span[class=price]")[0].text
    discount_price = t1[0].select("div[class*=product-price]")[0].findAll(name="span", attrs={"class":"price"})
except:
    discount_price = ""
#t1[0]
product_price

'94,00 TL'

In [6]:
for category_soup in restaurant_soup.select("div[class*=restaurantDetailBox]"):
    #try:
    for product_soup in category_soup.find(name="div", attrs={"class":"listBody"}).findAll(name="li"):
        print(product_soup.findAll(name="div", attrs={"class":"product-info"})[0].find(name="a", attrs={"class":"getProductDetail"}))
        #print(product_soup)
        break
    #except:
        #continue

<a class="getProductDetail" data-catalog-name="TR_ISTANBUL" data-category-name="cdc0d69d-4e9d-445e-8e62-d8d500e93888" data-product-id="559e852f-4256-45c5-9c98-c300ef06ccc4" data-top-sold-product="true" href="javascript:void(0);"> <i class="ys-icons ys-icons-foto" data-imagepath="//cdn.yemeksepeti.com/ProductImages/TR_ISTANBUL/pizzadante/pizzamista32cm._20210412193515_big.jpg" data-productname="Pizza Mista (32 cm.)"></i> </a>
<a class="getProductDetail" data-catalog-name="TR_ISTANBUL" data-category-name="cdc0d69d-4e9d-445e-8e62-d8d500e93888" data-product-id="DC93D4A3-F538-400F-A8D8-6E7F1DE13064" data-top-sold-product="false" href="javascript:void(0);">Seçilmiş Menü (Pizza Mediterraneo (25 cm.))</a>
<a class="getProductDetail" data-catalog-name="TR_ISTANBUL" data-category-name="cdc0d69d-4e9d-445e-8e62-d8d500e93888" data-product-id="4F9A1222-2174-4794-85BE-78B864137B58" data-top-sold-product="false" href="javascript:void(0);">Vodafone Menüsü (Pizza Refanzine (32 cm.))</a>
<a class="getPro